Финальный проект по временным рядам.

1. Мультивариативные ряды. Классификация или регрессия.
2. Один классический алгоритм.
3. Один НН алгоритм.

Мультивариативный ряд находится в папке data.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv("dry_bean_dataset/Dry_Bean_Dataset")